In [17]:

import lxml
from bs4 import BeautifulSoup
import folium  # map rendering library
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
import matplotlib.colors as colors
import matplotlib.cm as cm
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize
import requests  # library to handle requests
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim
import json  # library to handle JSON files
import pandas as pd  # library for data analsysis
import numpy as np  # library to handle data in a vectorized manner
print('Importing Libraries')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('.')
print('.')
# Matplotlib and associated plotting modules
# import k-means from clustering stage
print('.')
print('Done')


Importing Libraries
.
.
.
Done


In [18]:
# downloads and parses data (uses older version of wiki page for proper formatting)
r = requests.get(
    'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=876823784', timeout=5.00)
soup = BeautifulSoup(r.text, 'html.parser')
table = soup.find('table', attrs={'class': 'wikitable sortable'})

# retrieves page headers
headers = table.find_all('th')
for i, head in enumerate(headers):
    headers[i] = str(headers[i]).replace(
        "<th>", "").replace("</th>", "").replace("\n", "")


# finds all elements and skips first in table
rows = table.find_all('tr')
rows = rows[1:len(rows)]

# used to skip meta symbols and line feeds between rows
for i, row in enumerate(rows):
    rows[i] = str(rows[i]).replace(
        "\n</td></tr>", "").replace("<tr>\n<td>", "")

# creates dataframe, expands rows, and drops old row
df = pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n=2, expand=True)
df.drop(columns=[0], inplace=True)

# skip not assigned boroughs:
df = df.drop(df[(df.Borough == "Not assigned")].index)

# give "Not assigned" Neighborhoods same name as Borough:
df.Neighbourhood.replace("Not assigned", np.nan, inplace=True)


# copy Borough value to Neighborhood if NaN:
df.Neighbourhood.fillna(df.Borough, inplace=True)
# drop duplicate rows:
df = df.drop_duplicates()

# extract titles from columns
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)', expand=False))

df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)', expand=False))

# delete Toronto annotation from Neighbourhood:
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto", ""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)", ""))

# combine multiple neighborhoods with the same post code
df2 = pd.DataFrame({'Postcode': df.Postcode.unique()})
df2['Borough'] = pd.DataFrame(list(set(
    df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood'] = pd.Series(list(set(
    df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood'] = df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

df2.head()


C:\Users\16bre\AppData\Local\Temp/ipykernel_6544/479162430.py:57: FutureWarning: The default value of regex will change from True to False in a future version.
  df.Neighbourhood.loc[


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Queen's Park


In [19]:
# add Geo-spatial data
dfll = pd.read_csv("TO_geospatial_data.csv")
dfll.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df2.set_index("Postcode")
toronto_data = pd.merge(df2, dfll)
toronto_data.head()


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park (Toronto),Queen's Park,43.662301,-79.389494


In [20]:
dfcr = pd.read_csv("Neighbourhood_Crime_Rates_Boundary_File_.csv")
dfcr.rename(columns={'Neighbourhood' :'Neighborhood'}, inplace=True)
dfcr.set_index("Neighborhood")
toronto_data.set_index("Neighborhood")
toronto_data = pd.merge(toronto_data, dfcr)
toronto_data = pd.merge(toronto_data, dfll)
toronto_data.drop(['OBJECTID','Hood_ID'], axis=1, inplace=True)
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,Assault_AVG,Assault_CHG,Assault_Rate_2019,AutoTheft_2014,AutoTheft_2015,AutoTheft_2016,AutoTheft_2017,AutoTheft_2018,AutoTheft_2019,AutoTheft_AVG,AutoTheft_CHG,AutoTheft_Rate_2019,BreakandEnter_2014,BreakandEnter_2015,BreakandEnter_2016,BreakandEnter_2017,BreakandEnter_2018,BreakandEnter_2019,BreakandEnter_AVG,BreakandEnter_CHG,BreakandEnter_Rate_2019,Homicide_2014,Homicide_2015,Homicide_2016,Homicide_2017,Homicide_2018,Homicide_2019,Homicide_AVG,Homicide_CHG,Homicide_Rate_2019,Robbery_2014,Robbery_2015,Robbery_2016,Robbery_2017,Robbery_2018,Robbery_2019,Robbery_AVG,Robbery_CHG,Robbery_Rate_2019,TheftOver_2014,TheftOver_2015,TheftOver_2016,TheftOver_2017,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area,Shape__Length
0,M4A,North York,Victoria Village,43.725882,-79.315572,17510,118,138,133,83,112,132,119.3,0.18,753.9,20,14,20,14,13,18,16.5,0.38,102.8,38,46,23,32,35,60,39.0,0.71,342.7,0,0,2,1,0,1,0.7,1.0,5.7,10,18,13,10,14,14,13.2,0.00,80.0,6,6,4,5,4,5,5.0,0.25,28.6,4.755219e+06,11800.341701
1,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,14365,43,52,52,38,47,46,46.3,-0.02,320.2,11,7,17,22,24,16,16.2,-0.33,111.4,31,39,27,39,33,26,32.5,-0.21,181.0,0,1,0,1,0,0,0.3,0.0,0.0,8,11,8,6,4,10,7.8,1.50,69.6,3,3,5,6,7,4,4.7,-0.43,27.8,1.871263e+06,6036.268116
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,21567,83,108,86,111,94,81,93.8,-0.14,375.6,8,23,7,10,30,20,16.3,-0.33,92.7,44,42,77,33,43,57,49.3,0.33,264.3,0,0,0,0,0,0,0.0,0.0,0.0,15,14,19,48,10,16,20.3,0.60,74.2,8,5,3,7,5,9,6.2,0.80,41.7,3.595829e+06,11275.180743
3,M1G,"Scarborough, Toronto",Woburn,43.770992,-79.216917,53485,352,395,365,400,369,427,384.7,0.16,798.4,46,33,31,47,59,60,46.0,0.02,112.2,120,146,80,86,99,100,105.2,0.01,187.0,3,3,1,0,0,0,1.2,0.0,0.0,84,89,67,120,74,67,83.5,-0.09,125.3,11,13,14,23,13,8,13.7,-0.38,15.0,1.233407e+07,18111.264992
4,M1H,"Scarborough, Toronto",Woburn,43.773136,-79.239476,53485,352,395,365,400,369,427,384.7,0.16,798.4,46,33,31,47,59,60,46.0,0.02,112.2,120,146,80,86,99,100,105.2,0.01,187.0,3,3,1,0,0,0,1.2,0.0,0.0,84,89,67,120,74,67,83.5,-0.09,125.3,11,13,14,23,13,8,13.7,-0.38,15.0,1.233407e+07,18111.264992


In [21]:
dfin = pd.read_csv("wellbeing_toronto_incomedata.csv")
dfin.rename(columns={'Neighbourhood' :'Neighborhood'}, inplace=True)
dfin.set_index("Neighborhood")
dfin.drop('Neighbourhood Id', axis=1, inplace=True)
toronto_data = pd.merge(toronto_data, dfin)
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,Assault_AVG,Assault_CHG,Assault_Rate_2019,AutoTheft_2014,AutoTheft_2015,AutoTheft_2016,AutoTheft_2017,AutoTheft_2018,AutoTheft_2019,AutoTheft_AVG,AutoTheft_CHG,AutoTheft_Rate_2019,BreakandEnter_2014,BreakandEnter_2015,BreakandEnter_2016,BreakandEnter_2017,BreakandEnter_2018,BreakandEnter_2019,BreakandEnter_AVG,BreakandEnter_CHG,BreakandEnter_Rate_2019,Homicide_2014,Homicide_2015,Homicide_2016,Homicide_2017,Homicide_2018,Homicide_2019,Homicide_AVG,Homicide_CHG,Homicide_Rate_2019,Robbery_2014,Robbery_2015,Robbery_2016,Robbery_2017,Robbery_2018,Robbery_2019,Robbery_AVG,Robbery_CHG,Robbery_Rate_2019,TheftOver_2014,TheftOver_2015,TheftOver_2016,TheftOver_2017,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area,Shape__Length,Average Family Income
0,M4A,North York,Victoria Village,43.725882,-79.315572,17510,118,138,133,83,112,132,119.3,0.18,753.9,20,14,20,14,13,18,16.5,0.38,102.8,38,46,23,32,35,60,39.0,0.71,342.7,0,0,2,1,0,1,0.7,1.0,5.7,10,18,13,10,14,14,13.2,0.00,80.0,6,6,4,5,4,5,5.0,0.25,28.6,4.755219e+06,11800.341701,65104.0
1,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,14365,43,52,52,38,47,46,46.3,-0.02,320.2,11,7,17,22,24,16,16.2,-0.33,111.4,31,39,27,39,33,26,32.5,-0.21,181.0,0,1,0,1,0,0,0.3,0.0,0.0,8,11,8,6,4,10,7.8,1.50,69.6,3,3,5,6,7,4,4.7,-0.43,27.8,1.871263e+06,6036.268116,105770.0
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,21567,83,108,86,111,94,81,93.8,-0.14,375.6,8,23,7,10,30,20,16.3,-0.33,92.7,44,42,77,33,43,57,49.3,0.33,264.3,0,0,0,0,0,0,0.0,0.0,0.0,15,14,19,48,10,16,20.3,0.60,74.2,8,5,3,7,5,9,6.2,0.80,41.7,3.595829e+06,11275.180743,139757.0
3,M1G,"Scarborough, Toronto",Woburn,43.770992,-79.216917,53485,352,395,365,400,369,427,384.7,0.16,798.4,46,33,31,47,59,60,46.0,0.02,112.2,120,146,80,86,99,100,105.2,0.01,187.0,3,3,1,0,0,0,1.2,0.0,0.0,84,89,67,120,74,67,83.5,-0.09,125.3,11,13,14,23,13,8,13.7,-0.38,15.0,1.233407e+07,18111.264992,63186.0
4,M1H,"Scarborough, Toronto",Woburn,43.773136,-79.239476,53485,352,395,365,400,369,427,384.7,0.16,798.4,46,33,31,47,59,60,46.0,0.02,112.2,120,146,80,86,99,100,105.2,0.01,187.0,3,3,1,0,0,0,1.2,0.0,0.0,84,89,67,120,74,67,83.5,-0.09,125.3,11,13,14,23,13,8,13.7,-0.38,15.0,1.233407e+07,18111.264992,63186.0


In [22]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates for Toronto ON are latitude: {}, longitute: {}.'.format(
    latitude, longitude))


The geographical coordinates for Toronto ON are latitude: 43.6534817, longitute: -79.3839347.


In [23]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto


In [24]:
CLIENT_ID = 'GA5LNQMSFSRYYWKSU3FX4ITIVOFNSXXACIHKE3PAHSYI3DNB'  # your Foursquare ID
# your Foursquare Secret
CLIENT_SECRET = 'SFXBL03XIKKFYFMSBZTK1Q0GE2OXEGBZWWN4QIMMN32S3PT5'
VERSION = '20220409'  # Foursquare API version

print('Foursquare API Credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Foursquare API Credentials:
CLIENT_ID: GA5LNQMSFSRYYWKSU3FX4ITIVOFNSXXACIHKE3PAHSYI3DNB
CLIENT_SECRET:SFXBL03XIKKFYFMSBZTK1Q0GE2OXEGBZWWN4QIMMN32S3PT5


In [25]:
toronto_data.loc[0, 'Neighborhood']


'Victoria Village'

In [26]:
# neighborhood latitude value
neighborhood_latitude = toronto_data.loc[0, 'Latitude']
# neighborhood longitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude']

neighborhood_name = toronto_data.loc[0, 'Neighborhood']  # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name,
                                                               neighborhood_latitude,
                                                               neighborhood_longitude))


Latitude and longitude values of Victoria Village are 43.7258823, -79.3155716.


In [27]:
url = "https://api.foursquare.com/v3/places/search"

PARAMS = {
    'll':(str(neighborhood_latitude) + "," + str(neighborhood_longitude)),
    'radius': 500, # define radius
    'limit': 10  # limit of number of venues returned by Foursquare API
}

headers = {
    "Accept": "application/json",
    "Authorization": "fsq39l+XALKKrY9mbD/OXqED/cnk6KMe3P106YFn/CeTK9o="
}

response = requests.request("GET", url, headers=headers, params=PARAMS)
results = response.text
print(results)


{"results":[{"fsq_id":"4b61a2d9f964a520181b2ae3","categories":[{"id":17006,"name":"Car Dealership","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/automotive_","suffix":".png"}}],"chains":[{"id":"c45e3f83-2c4b-4ff7-ab74-58e2fade02b5","name":"Honda Dealer"}],"distance":89,"geocodes":{"main":{"latitude":43.726045,"longitude":-79.316691}},"link":"/v3/places/4b61a2d9f964a520181b2ae3","location":{"address":"1681 Eglinton Ave E","country":"CA","formatted_address":"1681 Eglinton Ave E, Toronto ON M4A 1J6","locality":"Toronto","neighborhood":["East York"],"postcode":"M4A 1J6","region":"ON"},"name":"Parkway Honda","related_places":{},"timezone":"America/Toronto"},{"fsq_id":"4b4b8becf964a520d39f26e3","categories":[{"id":17006,"name":"Car Dealership","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/automotive_","suffix":".png"}}],"chains":[{"id":"cbd69f0e-1352-498d-9911-ce3359556cbc","name":"Volkswagen Dealer"}],"distance":46,"geocodes":{"main":{"latitude":43.725923,"

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
venues = results['results']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

TypeError: string indices must be integers